In [ ]:
!pip install pinecone-client openai pypdf langchain langchain-community langchain-openai langchain-pinecone -qU pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
import pinecone
import os
import getpass
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_core.documents import Document
from uuid import uuid4

## Setup Pinecone

In [ ]:
pinecone_api = "pinecone_token"

In [ ]:
pc = Pinecone(api_key=pinecone_api)
spec = ServerlessSpec(cloud="aws", region="us-east-1")

In [ ]:
model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api)

In [ ]:
index_name = "coral-x"

In [ ]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.dimension,
        metric="cosine",
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [ ]:
try:
    indexes = pc.list_indexes()
    print("Successfully connected to Pinecone.")
    print("Existing indexes:", indexes)
except Exception as e:
    print(f"Failed to connect to Pinecone: {e}")

Successfully connected to Pinecone.
Existing indexes: [{
    "name": "coral-x",
    "dimension": 1024,
    "metric": "cosine",
    "host": "coral-x-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}, {
    "name": "rag-test",
    "dimension": 1024,
    "metric": "cosine",
    "host": "rag-test-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


In [ ]:
pc.describe_index(index_name)

{
    "name": "coral-x",
    "dimension": 1024,
    "metric": "cosine",
    "host": "coral-x-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [ ]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'data-context-docs': {'vector_count': 18},
                'global-context-docs': {'vector_count': 302}},
 'total_vector_count': 320}

In [ ]:
index.delete(delete_all=True, namespace='global-context-docs')

{}

In [ ]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'data-context-docs': {'vector_count': 18}},
 'total_vector_count': 18}

## Add data context docs

In [ ]:
namespace = "data-context-docs"

In [ ]:
vector_store = PineconeVectorStore(index = index, embedding = embeddings)

In [ ]:
path = "/content/drive/MyDrive/Capstone/rag_docs/data_context_docs"
files = os.listdir(path)
files

['state_stats.txt', 'purpose_wise_stats.txt', 'data_dict.txt']

In [ ]:
for file in files:
  with open(os.path.join(path, file)) as f:
    document = f.read()
  print(f"{file} read successfully.\n{document}Proceeding to chunk")
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunks = text_splitter.create_documents([document])
  print(f"Document chunked successfully")
  vector_store.add_documents(documents=chunks, namespace = namespace)
  print(f"Chunks uploaded to vector store\n")

state_stats.txt read successfully.
The following lines contain information about the median annual income, median loan amounts, median debt-to-income ratio and default rates for states in USA:
For the state of Alaska, the median annual income is $72000.0, the median loan amount is $15850.0, the median debt-to-income ratio is 18.395, and the loan default rate is 13.18%.
For the state of Alabama, the median annual income is $60441.520000000004, the median loan amount is $12150.0, the median debt-to-income ratio is 19.99, and the loan default rate is 15.76%.
For the state of Arkansas, the median annual income is $57000.0, the median loan amount is $12000.0, the median debt-to-income ratio is 20.19, and the loan default rate is 15.46%.
For the state of Arizona, the median annual income is $62000.0, the median loan amount is $12000.0, the median debt-to-income ratio is 17.97, and the loan default rate is 13.02%.
For the state of California, the median annual income is $70000.0, the median l

In [ ]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'data-context-docs': {'vector_count': 18}},
 'total_vector_count': 18}

## Add global context docs

In [ ]:
namespace = "global-context-docs"

In [ ]:
vector_store = PineconeVectorStore(index = index, embedding = embeddings)

In [ ]:
path = "/content/drive/MyDrive/Capstone/rag_docs/global_context_docs"
files = os.listdir(path)
files

['ch7.pdf',
 'iv-1-1.pdf',
 'Understanding the Common Reasons for Loan Rejections.pdf',
 '7 Reasons Why You Were Denied a Personal Loan | LendingTree.pdf']

In [ ]:
def create_docs(file_path):
  pdf_loader = PyPDFLoader(file_path)
  documents = pdf_loader.load()
  return documents

In [ ]:
def chunker(documents, chunk_size = 1000, chunk_overlap = 200):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  chunks = text_splitter.split_documents(documents)
  return chunks

In [ ]:
for file in files:
  documents = create_docs(os.path.join(path, file))
  print(f"{file} read successfully.\nProceeding to chunk")
  chunks = chunker(documents)
  print(f"Document chunked successfully")
  vector_store.add_documents(documents=chunks, namespace = namespace)
  print(f"Chunks uploaded to vector store\n")

ch7.pdf read successfully.
Proceeding to chunk
Document chunked successfully
Chunks uploaded to vector store

iv-1-1.pdf read successfully.
Proceeding to chunk
Document chunked successfully
Chunks uploaded to vector store

Understanding the Common Reasons for Loan Rejections.pdf read successfully.
Proceeding to chunk
Document chunked successfully
Chunks uploaded to vector store



7 Reasons Why You Were Denied a Personal Loan | LendingTree.pdf read successfully.
Proceeding to chunk
Document chunked successfully
Chunks uploaded to vector store



In [ ]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'data-context-docs': {'vector_count': 18},
                'global-context-docs': {'vector_count': 258}},
 'total_vector_count': 276}